# Decision Theory Project
*By Jelle Huibregtse and Aron Hemmes*

We will create a tic-tac-toe environment with the following characteristics.

Environment state:

- the agent is one of the nine squares
- an X or O can be placed on every empty square

In [75]:
from IPython.display import display, clear_output
import ipywidgets as widgets

## AI code

Next we have the code for the AI and the util used for it

In [76]:
# Count turns that have passed
def get_turns():
    global field
    n = 0
    for i in range(len(field)):
        if not field[i] == ' ':
            n += 1
    return n

# Check which player's turn it currently is
def get_turn():
    if get_turns() % 2 == 0:
        return 'X'
    return 'O'

# Calculate the best move for AI
def ai_move():
    global field, player, result
    
    ai = 'X'
    if player == 'X':
        ai = 'O'
    
    # If there's no result and it's the ai's turn
    if result == '' and get_turn() == ai:
        reward = 0
        

## Setup display

In [77]:
button_layout = widgets.Layout(width = '50px', height = '50px')

# Set field action
def player_move(e):
    global field, player, result
    
    # If there's no result and it's the player's turn
    if result == '' and get_turn() == player:
        # If field space isn't occupied 
        if field[int(e.tooltip)] == ' ':
            field[int(e.tooltip)] = player
            render()

# Reset field action
def reset(e):
    global field
    if not field == [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']:
        field = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
        render()
    elif e == '':
        render()

# Change player action
def player_change(e):
    global player
    if player == 'X':
        player = 'O'
    else:
        player = 'X'
    reset('')
    
def render():
    global field, player, result
    elements = []
    
    # Clear output
    clear_output(wait=True)
    
    # Calculate AI's move
    ai_move()
    
    # Add header
    elements.append(widgets.HTML(value="<h1>TicTacToe</h1>"))
    
    # Add field buttons
    buttons = []
    rows = []
    for i in range(9):
        btn = widgets.Button(tooltip = str(i), description = field[i], layout = button_layout)
        btn.on_click(player_move)
        buttons.append(btn)
        if (i + 1) % 3 == 0:
            rows.append(widgets.Box(buttons))
            buttons = []
    elements.append(widgets.Box(children = rows, layout = widgets.Layout(flex_flow = 'column')))
    
    # Add player select
    player_btn = widgets.Button(description = 'PLAYER ' + player, layout = widgets.Layout(width = '158px'))
    player_btn.on_click(player_change)
    
    elements.append(player_btn)
    
    # Add reset button
    reset_btn = widgets.Button(description = 'reset', layout = widgets.Layout(width = '158px'))
    reset_btn.on_click(reset)
    elements.append(reset_btn)
    
    # Display elements
    display(widgets.Box(children = elements, layout = widgets.Layout(flex_flow = 'column')))

## Start Game

Finally we can start the game

In [78]:
field = [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

result = ''

player = 'X'

render()

Box(children=(HTML(value='<h1>TicTacToe</h1>'), Box(children=(Box(children=(Button(description=' ', layout=Lay…